In [27]:
import numpy as np
import pandas as pd
from functools import reduce
import os
import statsmodels.api as sm

pd.set_option("precision",4)

# Market Data for US

In [28]:
df_CRSP_market_daily=pd.read_csv('../data/CRSP/CRSP Daily Index Built on Market Capitalization.csv',\
index_col=0,parse_dates=True)

In [29]:
df_CRSP_market_daily

,vwretd,veretx,ewretd,ewretx,totval,totcnt,usdval,usdcnt,spindx,sprtrn
Date,,,,,,,,,,
2000-01-03,-0.0068,-0.0068,0.0029,0.0029,16897673600,8358,17013544200,8357,1455.22,-0.0095
2000-01-04,-0.0397,-0.0397,-0.0175,-0.0175,16228867700,8359,16897669000,8357,1399.42,-0.0383
2000-01-05,-0.0009,-0.0010,0.0078,0.0077,16212787400,8360,16228817900,8356,1402.11,0.0019
2000-01-06,-0.0074,-0.0075,0.0045,0.0045,16089055800,8358,16199203000,8358,1403.45,0.0010
2000-01-07,0.0325,0.0325,0.0170,0.0170,16610768100,8353,16087694900,8353,1441.47,0.0271
...,...,...,...,...,...,...,...,...,...,...
2020-12-24,0.0023,0.0022,-0.0003,-0.0005,49387699400,7515,49278049200,7515,3703.06,0.0035
2020-12-28,0.0039,0.0039,0.0014,0.0013,49579392700,7516,49387699400,7514,3735.36,0.0087
2020-12-29,-0.0036,-0.0037,-0.0085,-0.0087,49396998600,7522,49576731700,7514,3727.04,-0.0022


In [30]:
df_CRSP_riskfree_monthly=pd.read_csv('../data/CRSP/CRSP Risk Free Rates.csv',\
index_col=4,parse_dates=True)

In [31]:
df_CRSP_riskfree_monthly

,KYTREASNOX,TIDXFAM,TTERMTYPE,TTERMLBL,RMTREASNO,RMCRSPID,TMBIDYTM,TMASKYTM,TMYTM,TMDURATN
MCALDT,,,,,,,,,,
2000-01-31,2000001,RISKFREE,30,CRSP Risk Free Rates - 1-Month (Nominal),203118,2.0000e+07,5.3900,5.3800,5.3900,31
2000-02-29,2000001,RISKFREE,30,CRSP Risk Free Rates - 1-Month (Nominal),203122,2.0000e+07,5.3200,5.3100,5.3100,30
2000-03-31,2000001,RISKFREE,30,CRSP Risk Free Rates - 1-Month (Nominal),203133,2.0001e+07,5.8600,5.8500,5.8600,34
2000-04-28,2000001,RISKFREE,30,CRSP Risk Free Rates - 1-Month (Nominal),203141,2.0001e+07,5.2800,5.2600,5.2700,34
2000-05-31,2000001,RISKFREE,30,CRSP Risk Free Rates - 1-Month (Nominal),203149,2.0001e+07,4.6800,4.6700,4.6700,36
...,...,...,...,...,...,...,...,...,...,...
2020-08-31,2000002,RISKFREE,90,CRSP Risk Free Rates - 3-Month (Nominal),207541,2.0201e+07,0.1060,0.0963,0.1010,88
2020-09-30,2000002,RISKFREE,90,CRSP Risk Free Rates - 3-Month (Nominal),207571,2.0201e+07,0.0913,0.0811,0.0862,90
2020-10-30,2000002,RISKFREE,90,CRSP Risk Free Rates - 3-Month (Nominal),207468,2.0210e+07,0.0913,0.0862,0.0887,90


In [32]:
df_AQR_risk_free_daily=pd.read_excel('../data/AQR Data/Betting Against Beta Equity Factors Daily.xlsx',\
sheet_name='RF',header=18,index_col=0,parse_dates=True)

In [33]:
df_AQR_risk_free_daily

,Risk Free Rate
DATE,
1926-07-31,1.0560e-04
1926-08-02,1.2000e-04
1926-08-03,1.2000e-04
1926-08-04,1.2000e-04
1926-08-05,1.2000e-04
...,...
2020-12-25,3.2000e-06
2020-12-28,3.2000e-06
2020-12-29,3.2000e-06


# Betas and Returns of Portfolio 

In [34]:
df_betas=pd.read_pickle('../data/Portfolio Beta and Return Daily/beta.pickle')
df_returns=pd.read_pickle('../data/Portfolio Beta and Return Daily/return.pickle')

In [35]:
df_betas.dropna()

,1,2,3,4,5,6,7,8,9,10
2007-12-31,0.4914,0.6835,0.7849,0.858,0.9174,0.9753,1.032,1.102,1.201,1.422
2008-01-02,0.4728,0.6628,0.7688,0.8466,0.909,0.9686,1.027,1.099,1.199,1.419
2008-01-03,0.4631,0.6552,0.7627,0.8414,0.9048,0.9657,1.025,1.097,1.197,1.417
2008-01-04,0.4589,0.6493,0.7581,0.8382,0.9026,0.9647,1.025,1.097,1.197,1.415
2008-01-07,0.4517,0.6428,0.7536,0.8339,0.8994,0.9624,1.023,1.095,1.194,1.412
...,...,...,...,...,...,...,...,...,...,...
2018-01-31,0.4378,0.6482,0.8852,1.049,1.176,1.293,1.41,1.547,1.75,2.336
2018-02-01,0.4384,0.6513,0.888,1.052,1.178,1.295,1.413,1.55,1.755,2.341
2018-02-02,0.4394,0.6466,0.885,1.05,1.177,1.295,1.411,1.55,1.754,2.342
2018-02-05,0.4365,0.6319,0.86,1.023,1.144,1.257,1.368,1.501,1.696,2.256


In [36]:
df_returns.dropna()

,1,2,3,4,5,6,7,8,9,10
2007-12-31,-0.02384,0.0009749,-0.001422,-0.002695,-0.004423,-0.00278,-0.005792,-0.006331,-0.005587,-0.003185
2008-01-02,0.01592,0.003656,-0.008629,-0.009899,-0.0109,-0.01269,-0.01509,-0.01398,-0.01672,-0.01615
2008-01-03,0.008987,0.001399,-0.006013,-0.008166,-0.008235,-0.01277,-0.01375,-0.01627,-0.01488,-0.001903
2008-01-04,0.005841,-0.01515,-0.01985,-0.02109,-0.02837,-0.03375,-0.0351,-0.03889,-0.03818,-0.05011
2008-01-07,0.003569,0.001342,0.002479,0.001888,0.0005488,0.002182,0.003867,-0.0001909,0.001077,0.002116
...,...,...,...,...,...,...,...,...,...,...
2018-01-31,0.003931,0.003143,0.001776,-0.0005346,-0.003801,-0.003886,-0.006153,-0.003675,-0.009557,-0.013
2018-02-01,-0.00414,-0.001954,-0.003858,-0.002224,-0.002026,0.0008792,0.0003772,0.001901,0.00043,-0.004098
2018-02-02,-0.007143,-0.00918,-0.01483,-0.01836,-0.02118,-0.01873,-0.02251,-0.02083,-0.02706,-0.03071
2018-02-05,-0.007768,-0.01257,-0.02626,-0.03167,-0.0338,-0.03387,-0.0361,-0.03466,-0.03503,-0.04064
